In [3]:
import os
import time
import contextlib
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras, feature_column
from sklearn import model_selection, preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow_hub as hub
import PIL.Image as Image
import tensorflow_datasets as tfds

from IPython.core.interactiveshell import InteractiveShell

In [2]:
# 配置项
# 这个要放到设置中文之前否则还是小方框
plt.style.use("seaborn")

# 指定默认字体 用来正常显示中文标签
plt.rcParams['font.sans-serif'] = ['SimHei']
# 解决保存图像是负号'-'显示为方块的问题
plt.rcParams['axes.unicode_minus'] = False

# #全部行都能输出
InteractiveShell.ast_node_interactivity = "all"

In [4]:
@contextlib.contextmanager
def assert_raises(error_class):
    """
    捕获异常的辅助代码
    """
    try:
        yield
    except error_class as e:
        print('Caught expected exception \n  {}: {}'.format(error_class, e))
    except Exception as e:
        print('Got unexpected exception \n  {}: {}'.format(type(e), e))
    else:
        raise Exception('Expected {} to be raised but no error was raised!'.format(
        error_class))



In [8]:
# function就像op
@tf.function
def add(a, b):
    return a+b

add(tf.ones([2, 2]), tf.ones([2, 2]))

<tf.Tensor: id=29, shape=(2, 2), dtype=float32, numpy=
array([[2., 2.],
       [2., 2.]], dtype=float32)>

In [10]:
# function 有对应的梯度
@tf.function
def add(a, b):
    return a + b

v = tf.Variable(1.0)
with tf.GradientTape() as t:
    result = add(v, 1.0)
# result = v+1, 对v求导result'(v) = 1
t.gradient(result, v)

<tf.Tensor: id=81, shape=(), dtype=float32, numpy=1.0>

In [14]:
# 还可以嵌套函数
@tf.function
def dense_layer(x, w, b):
    return add(tf.matmul(x, w), b)

dense_layer(tf.ones([3, 2]), tf.ones([2, 2]), tf.ones([2]))

<tf.Tensor: id=156, shape=(3, 2), dtype=float32, numpy=
array([[3., 3.],
       [3., 3.],
       [3., 3.]], dtype=float32)>

In [17]:
# tf.function的多样性
@tf.function
def double(a):
    print("Tracing with", a)
    return a + a

double(tf.constant(1))
print('\n')
double(tf.constant(1.1))
print('\n')
double(tf.Variable('a'))

Tracing with Tensor("a:0", shape=(), dtype=int32)


<tf.Tensor: id=231, shape=(), dtype=int32, numpy=2>



Tracing with Tensor("a:0", shape=(), dtype=float32)


<tf.Tensor: id=239, shape=(), dtype=float32, numpy=2.2>



Tracing with <tf.Variable 'Variable:0' shape=() dtype=string>


<tf.Tensor: id=255, shape=(), dtype=string, numpy=b'aa'>

In [20]:
print("Obtaining concrete trace")
double_strings = double.get_concrete_function(tf.TensorSpec(None, dtype=tf.string))
print("Executing traced function")
print(double_strings(tf.constant("a")))
print(double_strings(tf.constant("b")))

# 使用不匹配的类型就会抛出异常
print("Using a concrete trace with incompatible types will throw an error")
with assert_raises(tf.errors.InvalidArgumentError):
    double_strings(tf.constant(1))

Obtaining concrete trace
Executing traced function
tf.Tensor(b'aa', shape=(), dtype=string)
tf.Tensor(b'bb', shape=(), dtype=string)
Using a concrete trace with incompatible types will throw an error
Caught expected exception 
  <class 'tensorflow.python.framework.errors_impl.InvalidArgumentError'>: cannot compute __inference_double_261 as input #0(zero-based) was expected to be a string tensor but is a int32 tensor [Op:__inference_double_261]


In [24]:
# 调用graph时只追踪一次 就需要设置input_signature参数
@tf.function(input_signature=(tf.TensorSpec(shape=[None], dtype=tf.int32),))
def next_collatz(x):
    print('追踪 x', x)
    return tf.where(x % 2 == 0, x // 2, 3 * x + 1)

print(next_collatz(tf.constant([1, 2])))
with assert_raises(ValueError):
    next_collatz([[1, 2], [3, 4]])

追踪 x Tensor("x:0", shape=(None,), dtype=int32)
tf.Tensor([4 7], shape=(2,), dtype=int32)
Caught expected exception 
  <class 'ValueError'>: Python inputs incompatible with input_signature: inputs (([[1, 2], [3, 4]],)), input_signature ((TensorSpec(shape=(None,), dtype=tf.int32, name=None),))


In [25]:
def train_on_step():
    pass

@tf.function
def train(num_steps):
    print("Tracing with num_steps = {}".format(num_steps))
    for _ in range(num_steps):
        train_on_step()

# 这虽然会追踪,但是有个问题就是 当参数改变的时候又会在再次追踪 但是生成的图是相同的,这就有点浪费
train(num_steps=10)
train(num_steps=20)

Tracing with num_steps = 10
Tracing with num_steps = 20


In [26]:
# 这个方法较上面的好处就是 这个方法只会触发一次追踪 
# 解决方法:在不影响生成图的情况下,将参数转化为Tensor
train(num_steps=tf.constant(10))
train(num_steps=tf.constant(20))

Tracing with num_steps = Tensor("num_steps:0", shape=(), dtype=int32)


In [28]:
# tf.function 副作用
@tf.function
def f(x):
    print('Traced with', x)
    tf.print("Executed with", x)

f(1)
f(1)
f(3)

Traced with 1
Executed with 1
Executed with 1
Traced with 3
Executed with 3


In [33]:
# 你如果在每次触发function的时候都触发Python有关的代码 那么你就需要使用py_function
# 注意这段代码和上段代码的区别 这段代码print函数打印了3次 上面的代码print打印了两次
# 或者直接使用tf的方法进行观察 eg:tf.Variable.assign, tf.print, and tf.summary
external_list = []
def side_effect(x):
  print('Python side effect')
  external_list.append(x)

    
@tf.function
def f(x):
    tf.py_function(side_effect, inp=[x], Tout=[])

f(1.0)
f(1.0)
f(3.0)

assert len(external_list) == 3
# .numpy() call required because py_function casts 1 to tf.constant(1)
assert external_list[0].numpy() == 1

Python side effect
Python side effect
Python side effect


In [39]:
# 使用tf.function在生成器和迭代器中会有意想不到的事情发生
external_var = tf.Variable(0)

@tf.function
def buggy_consume_next(iterator):
    external_var.assign_add(next(iterator))
    tf.print("Value of external_var:", external_var)

iterator = iter([0, 1, 2, 3])
# 本来这应该返回0, 1, 2, 3
buggy_consume_next(iterator)
buggy_consume_next(iterator)
buggy_consume_next(iterator)
buggy_consume_next(iterator)

Value of external_var: 0
Value of external_var: 0
Value of external_var: 0
Value of external_var: 0


In [40]:
external_var = tf.Variable(0)

@tf.function
def buggy_consume_next(iterator):
    iterator = iter([0, 1, 2, 3])
    external_var.assign_add(next(iterator))
    tf.print("Value of external_var:", external_var)

iterator = iter([0, 1, 2, 3])
# 本来这应该返回0, 1, 2, 3
buggy_consume_next(iterator)
buggy_consume_next(iterator)
buggy_consume_next(iterator)
buggy_consume_next(iterator)

Value of external_var: 0
Value of external_var: 0
Value of external_var: 0
Value of external_var: 0


In [41]:
# 解决上面问题的方案就是把生成器和迭代器放到@tf.function  不是很明白???
def measure_graph_size(f, *args):
  g = f.get_concrete_function(*args).graph
  print("{}({}) contains {} nodes in its graph".format(
      f.__name__, ', '.join(map(str, args)), len(g.as_graph_def().node)))

@tf.function
def train(dataset):
  loss = tf.constant(0)
  for x, y in dataset:
    loss += tf.abs(y - x) # Some dummy computation.
  return loss

small_data = [(1, 1)] * 2
big_data = [(1, 1)] * 10
measure_graph_size(train, small_data)
measure_graph_size(train, big_data)

measure_graph_size(train, tf.data.Dataset.from_generator(
    lambda: small_data, (tf.int32, tf.int32)))

measure_graph_size(train, tf.data.Dataset.from_generator(
    lambda: big_data, (tf.int32, tf.int32)))


W0823 15:17:09.961825 4669924800 deprecation.py:323] From /anaconda3/lib/python3.7/site-packages/tensorflow/python/data/ops/dataset_ops.py:504: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


train([(1, 1), (1, 1)]) contains 8 nodes in its graph
train([(1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1)]) contains 32 nodes in its graph
train(<DatasetV1Adapter shapes: (<unknown>, <unknown>), types: (tf.int32, tf.int32)>) contains 4 nodes in its graph
train(<DatasetV1Adapter shapes: (<unknown>, <unknown>), types: (tf.int32, tf.int32)>) contains 4 nodes in its graph


In [44]:
# Automatic control dependencies

a = tf.Variable(1.0)
b = tf.Variable(2.0)

@tf.function
def f(x, y):
  # a = 4
  a.assign(y * b)
  # b = b + x * a = 2 + 4 = 6
  b.assign_add(x * a)

  return a + b

f(1.0, 2.0)  # 10.0

<tf.Tensor: id=808, shape=(), dtype=float32, numpy=10.0>

In [45]:
# 每次调用时都会创建一个一个变量 这样的话会报错
@tf.function
def f(x):
  v = tf.Variable(1.0)
  v.assign_add(x)
  return v

with assert_raises(ValueError):
  f(1.0)

Caught expected exception 
  <class 'ValueError'>: in converted code:

    <ipython-input-45-73e410646579>:3 f  *
        v = tf.Variable(1.0)
    /anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/variables.py:262 __call__
        return cls._variable_v2_call(*args, **kwargs)
    /anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/variables.py:256 _variable_v2_call
        shape=shape)
    /anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/variables.py:60 getter
        return captured_getter(captured_previous, **kwargs)
    /anaconda3/lib/python3.7/site-packages/tensorflow/python/eager/def_function.py:364 invalid_creator_scope
        "tf.function-decorated function tried to create "

    ValueError: tf.function-decorated function tried to create variables on non-first call.



In [47]:
v = tf.Variable(1.0)

@tf.function
def f(x):
    v.assign_add(x)
    return v

f(1.0)
f(2.0)

<tf.Tensor: id=876, shape=(), dtype=float32, numpy=2.0>

<tf.Tensor: id=884, shape=(), dtype=float32, numpy=4.0>

In [48]:
# tf.function 中创建变量也是可以的 只要是第一次创建就可以 如下面这个例子中obj.v
class C: pass
obj = C(); obj.v = None

@tf.function
def g(x):
  if obj.v is None:
    obj.v = tf.Variable(1.0)
  return obj.v.assign_add(x)

print(g(1.0))  # 2.0
print(g(2.0))  # 4.0

tf.Tensor(2.0, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)


In [49]:
state = []
@tf.function
def fn(x):
  if not state:
    state.append(tf.Variable(2.0 * x))
    state.append(tf.Variable(state[0] * 3.0))
  return state[0] * x * state[1]

print(fn(tf.constant(1.0)))
print(fn(tf.constant(3.0)))

tf.Tensor(12.0, shape=(), dtype=float32)
tf.Tensor(36.0, shape=(), dtype=float32)


### Using AutoGraph


In [52]:
@tf.function
def f(x):
    while tf.reduce_sum(x) > 1:
        tf.print(x)
        x = tf.tanh(x)
    return x
f(tf.random.uniform([5]))

[0.197801948 0.2193712 0.572006226 0.529604435 0.701354623]
[0.195261985 0.215918645 0.51683116 0.485078663 0.605226934]
[0.192817643 0.21262458 0.475250542 0.450301707 0.540758491]
[0.190463066 0.209477261 0.442431867 0.422146976 0.493561924]
[0.188192889 0.206466094 0.415658206 0.398737609 0.457038939]
[0.18600218 0.203581482 0.393266439 0.378868312 0.427667797]
[0.183886394 0.200814813 0.374172747 0.361724228 0.40337038]
[0.181841373 0.198158249 0.357636124 0.346731842 0.3828291]
[0.179863244 0.195604667 0.343130112 0.333474 0.365161836]
[0.17794843 0.19314757 0.330268949 0.321638882 0.349752545]
[0.176093623 0.190781 0.318762392 0.310988039 0.33615607]
[0.174295738 0.188499525 0.308387399 0.301335663 0.324041337]
[0.17255193 0.186298192 0.298969328 0.292534411 0.313156515]
[0.17085956 0.184172392 0.290369093 0.28446579 0.303305954]
[0.169216082 0.182117909 0.282474488 0.27703318 0.294335037]
[0.167619228 0.180130824 0.275193721 0.270157 0.286119848]
[0.16606684 0.178207532 0.268450

<tf.Tensor: id=1110, shape=(5,), dtype=float32, numpy=
array([0.1537989 , 0.16328673, 0.22409761, 0.22135927, 0.22986391],
      dtype=float32)>

In [57]:
# 如果你感兴趣你可以查看代码autograph的生成, 他看起来像是汇编
def f(x):
    while tf.reduce_sum(x) > 1:
        tf.print(x)
        x = tf.tanh(x)
    return x
print(tf.autograph.to_code(f))


"def tf__f(x):\n  do_return = False\n  retval_ = ag__.UndefinedReturnValue()\n\n  def loop_test(x_1):\n    return ag__.converted_call('reduce_sum', tf, ag__.ConversionOptions(recursive=True, force_conversion=False, optional_features=(), internal_convert_user_code=True), (x_1,), None) > 1\n\n  def loop_body(x_1):\n    ag__.converted_call('print', tf, ag__.ConversionOptions(recursive=True, force_conversion=False, optional_features=(), internal_convert_user_code=True), (x_1,), None)\n    x_1 = ag__.converted_call('tanh', tf, ag__.ConversionOptions(recursive=True, force_conversion=False, optional_features=(), internal_convert_user_code=True), (x_1,), None)\n    return x_1,\n  x, = ag__.while_stmt(loop_test, loop_body, (x,))\n  do_return = True\n  retval_ = x\n  cond = ag__.is_undefined_return(retval_)\n\n  def get_state():\n    return ()\n\n  def set_state(_):\n    pass\n\n  def if_true():\n    retval_ = None\n    return retval_\n\n  def if_false():\n    return retval_\n  retval_ = ag__.if

def tf__f(x):
  do_return = False
  retval_ = ag__.UndefinedReturnValue()

  def loop_test(x_1):
    return ag__.converted_call('reduce_sum', tf, ag__.ConversionOptions(recursive=True, force_conversion=False, optional_features=(), internal_convert_user_code=True), (x_1,), None) > 1

  def loop_body(x_1):
    ag__.converted_call('print', tf, ag__.ConversionOptions(recursive=True, force_conversion=False, optional_features=(), internal_convert_user_code=True), (x_1,), None)
    x_1 = ag__.converted_call('tanh', tf, ag__.ConversionOptions(recursive=True, force_conversion=False, optional_features=(), internal_convert_user_code=True), (x_1,), None)
    return x_1,
  x, = ag__.while_stmt(loop_test, loop_body, (x,))
  do_return = True
  retval_ = x
  cond = ag__.is_undefined_return(retval_)

  def get_state():
    return ()

  def set_state(_):
    pass

  def if_true():
    retval_ = None
    return retval_

  def if_false():
    return retval_
  retval_ = ag__.if_stmt(cond, if_true, if_false

### AutoGraph: Conditionals

In [58]:
def test_tf_cond(f, *args):
    g = f.get_concrete_function(*args).graph
    if any(node.name == 'cond' for node in g.as_graph_def().node):
        print("{}({}) uses tf.cond.".format(
        f.__name__, ', '.join(map(str, args))))
    else:
        print("{}({}) executes normally.".format(
        f.__name__, ', '.join(map(str, args))))

In [59]:
@tf.function
def hyperparam_cond(x, training=True):
  if training:
    x = tf.nn.dropout(x, rate=0.5)
  return x

@tf.function
def maybe_tensor_cond(x):
  if x < 0:
    x = -x
  return x

# 这个没有看明白 什么样的情况下node.name == 'cond'
test_tf_cond(hyperparam_cond, tf.ones([1], dtype=tf.float32))
test_tf_cond(maybe_tensor_cond, tf.constant(-1))
test_tf_cond(maybe_tensor_cond, -1)

hyperparam_cond(tf.Tensor([1.], shape=(1,), dtype=float32)) executes normally.
maybe_tensor_cond(tf.Tensor(-1, shape=(), dtype=int32)) uses tf.cond.
maybe_tensor_cond(-1) executes normally.


In [60]:
# Tracing both sides can result in unexpected execution of Python code - 
# it requires that if one branch creates a tensor used downstream, 
# the other branch must also create that tensor.
# 追踪两边会发生异常 必须要求if分支用downstream创建tensor, 另外一个分支也必须创建tensor
# 追踪两边会发生异常 例如下面这个 if...else里面的代码都走了
@tf.function
def f():
    x = tf.constant(1)
    if tf.constant(True):
        x = x + 1
        print('Tracing `then` branch')
    else:
        x = x - 1
        print('Tracing `else` branch')
    return x
f()

Tracing `then` branch
Tracing `else` branch


<tf.Tensor: id=1185, shape=(), dtype=int32, numpy=2>

In [61]:
def test_dynamically_unrolled(f, *args):
  g = f.get_concrete_function(*args).graph
  if any(node.name == 'while' for node in g.as_graph_def().node):
    print("{}({}) uses tf.while_loop.".format(
        f.__name__, ', '.join(map(str, args))))
  elif any(node.name == 'ReduceDataset' for node in g.as_graph_def().node):
    print("{}({}) uses tf.data.Dataset.reduce.".format(
        f.__name__, ', '.join(map(str, args))))
  else:
    print("{}({}) gets unrolled.".format(
        f.__name__, ', '.join(map(str, args))))

In [62]:
@tf.function
def for_in_range():
  x = 0
  for i in range(5):
    x += i
  return x

test_dynamically_unrolled(for_in_range)

for_in_range() gets unrolled.


In [67]:
# 这段代码和上面的区别就在tf.range
@tf.function
def for_in_tfrange():
  x = tf.constant(0, dtype=tf.int32)
  for i in tf.range(5):
    x += i
  return x

test_dynamically_unrolled(for_in_tfrange)

for_in_tfrange() uses tf.while_loop.


In [68]:
@tf.function
def for_in_tfdataset():
  x = tf.constant(0, dtype=tf.int64)
  for i in tf.data.Dataset.range(5):
    x += i
  return x

test_dynamically_unrolled(for_in_tfdataset)

for_in_tfdataset() uses tf.data.Dataset.reduce.


In [69]:
@tf.function
def while_py_cond():
  x = 5
  while x > 0:
    x -= 1
  return x

test_dynamically_unrolled(while_py_cond)

In [71]:
# 这段代码和下面的代码就在于变量x的定义不同,一个是tensor 一个是普通变量
@tf.function
def while_tf_cond():
  x = tf.constant(5)
  while x > 0:
    x -= 1
  return x


test_dynamically_unrolled(while_tf_cond)

while_tf_cond() uses tf.while_loop.


In [70]:
@tf.function
def while_py_true_py_break(x):
  while True:  # py true
    if x == 0: # py break
      break
    x -= 1
  return x

test_dynamically_unrolled(while_py_true_py_break, 5)

while_py_true_py_break(5) gets unrolled.


In [72]:
# 上下两端代码的区别在于 while的循环条件略有不同, tf.constant(True) / True
@tf.function
def while_tf_true_tf_break(x):
  while tf.constant(True): # tf true
    if x == 0:  # py break
      break
    x -= 1
  return x

test_dynamically_unrolled(while_tf_true_tf_break, 5)

while_tf_true_tf_break(5) uses tf.while_loop.
